In [ ]:
import spacy.cli
spacy.cli.download("en_core_web_md")

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [26]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import spacy
Range=[15,30]
nlp=spacy.load('en_core_web_md')
df=pd.read_csv("data_100_input.csv")
df.head()

,keyword,title 1,description 1,title 2,description 2,title 3,description 3,title 4,description 4,title 5,description 5
0,how to train a puppy,Crates Rank High as a Potty Training Tool,Many people who are new to dogs cringe at the ...,Puppy Pads and Paper Training,Dr. Burch says the use of puppy pads and paper...,Create a Housetraining Schedule for Your Puppy,It is vital to housetraining success. Puppies ...,Train your puppy: To use a crate,"It’s not easy learning how to train a puppy, b...",Train your puppy: To not bite,Puppies like to nip a lot while their teeny-ti...
1,how to stop a puppy from biting,Teach your puppy bite inhibition,Learning how to moderate the force of a bite i...,Teach your puppy that biting means “game over”,"If your puppy bites you while playing, that me...",Inhibit the Biting,The first step for how to get your puppy to st...,Redirect,If you’d like to teach your puppy that her mou...,Distraction,"In addition to mouthing people, puppies will a..."
2,how to crate train a puppy,Where Should I Keep My Dog's Crate?,The perfect spot for a dog crate is a location...,How to Crate Train Your Dog: Tips and Tricks,The following simple tips can help speed the c...,The Right Crate,Once you start to look at crates you will find...,Reward your puppy and create a positive experi...,The best way to find success in crate training...,Crate Training A Puppy At Night,"Despite your best efforts, the first night may..."
3,how to potty train a puppy,Get your puppy on a feeding schedule,One good way to get pottying under control is ...,MORE BLOGS FROM RICHELL USA!,The Importance of a Clean Litter Box\n\n12.30....,Before you start: Adopt the right mindset,"To guide your potty training journey, it’s hel...",How long does puppy potty training take?,The time it takes to fully potty train your pu...,How Long Does It Take To Potty Train A Labrado...,"Just like Golden Retrievers, we have a lot of ..."
4,how much to feed a puppy,WHAT TO FEED YOUR PUPPY,Before jumping into how much food you need to ...,SIZE OF THE BREED,A large breed puppy will need a large breed‐sp...,TYPE OF FOOD,"When it comes to feeding your puppy food, dry ...",Puppy Feeding Tidbits,• Feeding your pup the moment you get home may...,Puppy Feeding Advice,Make sure everyone gets with the program\n\nYo...


In [3]:
!pip install sentence-splitter

In [4]:
!pip install transformers

In [4]:
!pip install SentencePiece

     |████████████████████████████████| 1.2 MB 5.1 MB/s 


In [3]:
import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

model_name = 'tuner007/pegasus_paraphrase'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

def get_response(input_text):
    batch = tokenizer.prepare_seq2seq_batch([input_text],truncation=True,padding='longest',max_length=60, return_tensors="pt").to(torch_device)
    translated = model.generate(**batch,max_length=60,num_beams=10, num_return_sequences=1, temperature=1.5)
    tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
    return tgt_text

    

In [24]:
def paraphrase_article(cell_text,Range):
    article=[]
    for ind,i in enumerate(cell_text.split("\n")):
        if len(i)>3:
            text=nlp(i)
            paraphrased_text_list=[]
            for sent in text.sents:
                paraphrased_text=get_response(sent.text.strip(".,'"))
                paraphrased_text_list.append(paraphrased_text[0])
            paraphrased_paragraph=' '.join(paraphrased_text_list)
            #if ind!=0 and ind!=len(cell_text.split('\n'))-1:
            #    if len(i) in range(Range[0],Range[1]+1):
            #        paraphrased_paragraph="<h3>"+paraphrased_paragraph+"</h3>"
            if(paraphrased_paragraph.count('800')>=2 or paraphrased_paragraph.count('888')>=2) or paraphrased_paragraph.count('1-65561')>=2:
                article.append(i)
            else:
                article.append(paraphrased_paragraph)
        else:
            article.append('')

    return '\n'.join(article)


In [25]:
print(torch.cuda.is_available())

True


In [4]:
paraphrased_df=pd.DataFrame()
for i,col in enumerate(df.columns.to_list()):
    if 'description' in col: 
        text_list=[]
        for text in df[col]:
            output=paraphrase_article(text,Range)
            
            text_list.append(output)
        paraphrased_df[col]=text_list
    else:
        paraphrased_df[col]=df[col]

In [5]:
paraphrased_df.to_csv('formated_output_4.csv')
paraphrased_df.to_excel('formated_output_4.xlsx')